In [1]:
from gensim.models import Phrases

In [7]:
import string
import sys
import io 
import nltk

# Stopwords
from gensim.parsing.preprocessing import remove_stopwords
from nltk.corpus import stopwords 
nltk.download('stopwords')
stops = set(stopwords.words('italian'))
nltk_stopwords = nltk.corpus.stopwords.words('italian')
# Valutare di aggiungere eventuali forme arcariche delle stopwords italiane, o se c'è necessità di estendere questa lista
  
# Divido il testo in frasi in base ai punti
nltk.download('punkt')

# Tokenizer
from nltk.tokenize import word_tokenize 

# Per il discorso lemmatizzazione dobbiamo valutare come muoverci con l'italiano

# Lemmatization
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# Position tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Contatore parole uniche
from collections import Counter

# Per esplorare risultati
import random

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cafe6\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cafe6\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cafe6\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cafe6\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cafe6\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
from gensim.models.phrases import Phrases, Phraser
def build_phrases(sentences):
    phrases = Phrases(sentences,
                      min_count=7,
                      threshold=10,
                      progress_per=1000)
    return Phraser(phrases)

In [4]:
import os
os.chdir('Books')

In [5]:
# Se non ho capito male, per addestrare il Phraser che identifica i bigrammi ho bisogno del testo processato, ossia la lista di
# liste contenenti le frasi. Quindi carico la funzione di preprocessing

def preprocessing(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze, ...
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frasi, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    filtered_sentence = [w for w in noalfa_sentence if ((w not in stops) and (len(w) > 1))]
    # Ricostruisco la lista con le frasi 'pulite'
    if filtered_sentence:
      filtered_sentences.append(filtered_sentence)

  return filtered_sentences

In [8]:
frasi_decameron = preprocessing('Decameron.txt')

In [9]:
# Istruisco il modello Phraser, e lo salvo per non doverlo istruirlo ogni volta
phrases_model = build_phrases(frasi_decameron)
phrases_model.save('phrases_model.txt')

In [10]:
# Questa funzione ricostruisce la frase con i bigrammi
def sentence_to_bi_grams(phrases_model, sentence):
    return ' '.join(phrases_model[sentence])

In [11]:
# Esempio di funzionamento
sentence_to_bi_grams(phrases_model, frasi_decameron[0])

'comincia libro chiamato decameron cognominato prencipe galeotto contengono cento novelle dieci dì dette sette donne tre_giovani uomini'

In [18]:
# Adatto la funzione di preprocessing per inserire anche i bigrammi

def preprocessing_w_bigrams(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze, ...
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frasi, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    filtered_sentence = [w for w in noalfa_sentence if ((w not in stops) and (len(w) > 1))]
    # Valuto i bigrammi
    bigram_sentence = sentence_to_bi_grams(phrases_model, filtered_sentence)
    bigram_sentence = bigram_sentence.split() # Rimetto tutto in token
    # Ricostruisco la lista con le frasi 'pulite'
    if bigram_sentence:
      filtered_sentences.append(bigram_sentence)

  return filtered_sentences

In [19]:
frasi_decameron_bigrams = preprocessing_w_bigrams("Decameron.txt")

In [20]:
frasi_decameron_bigrams

[['comincia',
  'libro',
  'chiamato',
  'decameron',
  'cognominato',
  'prencipe',
  'galeotto',
  'contengono',
  'cento',
  'novelle',
  'dieci',
  'dì',
  'dette',
  'sette',
  'donne',
  'tre_giovani',
  'uomini'],
 ['proemio',
  'umana',
  'cosa',
  'aver',
  'compassione',
  'afflitti',
  'ciascuna',
  'persona',
  'stea',
  'bene',
  'coloro',
  'massimamente',
  'richiesto',
  'quali',
  'già',
  'conforto',
  'mestiere',
  'hannol',
  'trovato',
  'alcuni',
  'fra',
  'quali',
  'alcuno',
  'mai',
  'bisogno',
  'caro',
  'già',
  'ricevette',
  'piacere',
  'quegli'],
 ['ciò',
  'prima',
  'giovanezza',
  'infino',
  'tempo',
  'oltre_modo',
  'acceso',
  'stato',
  'nobile',
  'amore',
  'forse',
  'assai',
  'bassa_condizione',
  'parrebbe',
  'narrandolo',
  'richiedesse',
  'quantunque',
  'appo',
  'coloro',
  'discreti',
  'notizia',
  'pervenne',
  'lodato',
  'molto',
  'reputato',
  'nondimeno',
  'egli',
  'grandissima',
  'fatica',
  'sofferire',
  'certo',
  'cr

In [21]:
# Provo a vedere se funziona Word2Vec

import os
from gensim.models import Word2Vec
from collections import Counter

In [22]:
model = Word2Vec(sentences=frasi_decameron_bigrams,
                    #window = 5, default value
                    min_count=10, #not consider word with absolute frequency <10 
                    vector_size=300, #vector size 
                    sg = 1, #skipgram algorithm
                    hs = 0,
                    negative = 5, #negative sampling with 5 noise words
                    workers = 5, #faster process
                    epochs = 6 #6 iterations
                    )

In [24]:
model.wv.most_similar("amore")

[('amor', 0.9862819910049438),
 ('niuna_cosa', 0.9807226657867432),
 ('cara', 0.9794481992721558),
 ('disidero', 0.9788221716880798),
 ('altrui', 0.978757917881012),
 ('noia', 0.9783289432525635),
 ('potrebbe', 0.976768434047699),
 ('morire', 0.9760075211524963),
 ('niun', 0.9757950305938721),
 ('però', 0.9757428169250488)]